In [1]:
from google.colab import files
uploaded = files.upload()


Saving archive.zip to archive.zip


In [2]:
import zipfile, os

for file_name in uploaded.keys():
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall('dataset')



In [3]:
!ls dataset


CEAS_08.csv  Ling.csv	  Nigerian_Fraud.csv  SpamAssasin.csv
Enron.csv    Nazario.csv  phishing_email.csv


In [4]:
import pandas as pd

data = pd.read_csv("dataset/phishing_email.csv")
data.head()


,text_combined,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0


In [5]:
data.columns


Index(['text_combined', 'label'], dtype='object')

In [7]:
import re

def clean_text(text):
    text = re.sub(r"http\S+", "", str(text))
    text = re.sub(r"[^a-zA-Z ]", "", text)
    text = text.lower()
    return text

data['text_combined'] = data['text_combined'].apply(clean_text)


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = data['text_combined']
y = data['label']

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(X)


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [11]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9814522972481513
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      7935
           1       0.98      0.99      0.98      8563

    accuracy                           0.98     16498
   macro avg       0.98      0.98      0.98     16498
weighted avg       0.98      0.98      0.98     16498



In [12]:
sample = ["Your PayPal account is limited. Verify now to restore access."]
sample = vectorizer.transform(sample)

prediction = model.predict(sample)

print("🚨 Phishing Detected" if prediction[0]==1 else "✅ Safe Email")


🚨 Phishing Detected


In [13]:
import joblib

joblib.dump(model, "phishing_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")


['vectorizer.pkl']

In [14]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.9 MB/s eta 0:00:00


In [15]:
%%writefile app.py
import streamlit as st
import joblib
import re

model = joblib.load("phishing_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

def clean_text(text):
    text = re.sub(r"http\S+", "", str(text))
    text = re.sub(r"[^a-zA-Z ]", "", text)
    text = text.lower()
    return text

st.title("🛡️ AI Phishing Detection System")

email = st.text_area("Paste email text here:")

if st.button("Check Email"):
    email = clean_text(email)
    email_vec = vectorizer.transform([email])
    prediction = model.predict(email_vec)

    if prediction[0] == 1:
        st.error("🚨 This is a Phishing Email")
    else:
        st.success("✅ This is a Safe Email")


Writing app.py


In [18]:
!ngrok config add-authtoken 2zVbNQYZ65Nx7fnoy0kprvKLFpc_2toJJ1utf2jpsnn3qYw14


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
!streamlit run app.py &>/content/logs.txt &


In [20]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://85c3e6b1fa0a.ngrok-free.app" -> "http://localhost:8501">